### SSH setup

In [94]:
# import the modules
import paramiko
from getpass import getpass
import os


# Check to see if result text files exists
rootdir = 'results'
fList = []
# Crawl through the provided directory
for root, subfolders, filenames in os.walk(rootdir):
    for f in filenames:
        fileList = root + "/" + f
        fList.append(fileList)

import paramiko
from getpass import getpass
import os, os.path

# Check to see if ssh_results file exists
if os.path.exists('ssh_results.txt'):
    # Ask user to delete the file (not overwrite)
    user_input = input("There is a file under that name already. Want to delete it, sire? (y/n) ")
    # If yes, deletes the file.
    if user_input == "y":
        os.remove('ssh_results.txt')



# Creates the password prompt
thePass = getpass(prompt="Please enter your SSH password.")

# Host information
host = "192.168.6.71"
port = 2222
username = "john.tiseo"
password = thePass

try :
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host,port,username,password)
except paramiko.AuthenticationException:
    print ("Authentication failed")

### I figured out I can just put python functions in the book here, so this is a function that runs preset command lists and then saves the results to a file.


In [95]:
def runCMDList(commandLists, passwd, outFile):

    for eachCMD in commandLists:

        # Get the output from the command
        stdin, stdout, stderr = ssh.exec_command(eachCMD)
        # Password input
        stdin.write(passwd + "\n")
        # Get results from stdout command
        lines = stdout.readlines()
        # Convert list to string
        output = ''.join(lines)
        # Header
        sepHeader = '' + '### BEGIN ' + eachCMD + ' ###\n\n'
        # Footer
        sepFooter = '' + '### END ' + eachCMD  + ' ###\n\n'
        # Concatenates the header, command, footer
        cmd_output = sepHeader + output + sepFooter
        # Saves cmd_output to a file
        with open(outFile, 'a') as f:
            f.write(cmd_output)
        # Print results
        print(cmd_output)

In [96]:
#Downloads file from remote to host systems
def downloadFiles(files, downloadDirectory):
    # Going through and downloading
    for eachFile in files:
        # Filename list
        x = eachFile.split("/")
        # -1 for last element in order
        filename = x[-1]
        # Local path for file storage
        local_path = str(downloadDirectory) +  "/" +str(filename)
        # File grab
        sftp.get(eachFile,local_path)


### Upload Kranken and run it against /usr/bin/, /usr/sbin, /usr/local/bin, and /usr/local/sbin

In [97]:
# start SFTP session
sftp = ssh.open_sftp()
# the local file on the system to upload
localFile = "kraken"
# the remote path to upload the file to
remotePath = "/home/john.tiseo/kraken"
# Push file from server to our path specified
sftp.put(localFile, remotePath)
# Run commands, so we own kraken and our folders
command = ['sudo -S chmod +x kraken','sudo -S /home/john.tiseo/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin']
# File for output
output = "krakenR.txt"
# Run commands
runCMDList(command,thePass,output)

### BEGIN sudo -S chmod +x kraken ###

### END sudo -S chmod +x kraken ###


### BEGIN sudo -S /home/john.tiseo/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin ###

INFO[0000] Looking for configuration file with name config.yaml 
INFO[0000] No configuration file found, generating a default one... 
INFO[0000] Loading Yara rules...                        
INFO[0000] Scanning running processes...                
WARN[0004] DETECTION! Malicious process detected as foundGoBinary  pid=3138 process=ls
WARN[0005] DETECTION! Malicious process detected as foundGoBinary  pid=25680 process=snapd
INFO[0016] Scanning autoruns...                         
WARN[0017] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
WARN[0017] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
INFO[0018] Scanning the filesystem (this can take se

### Look for malicious PIDs we found in last step

In [98]:
# Our sus PID
pid = 3138
# Command execute
command = ["sudo -S lsof -p" + str(pid)]
# File for output
output = "lsofR.txt"
runCMDList(command,thePass,output)

### BEGIN sudo -S lsof -p3138 ###

COMMAND  PID     USER   FD      TYPE DEVICE SIZE/OFF   NODE NAME
ls      3138 p.dalton  cwd       DIR    8,1     4096 258130 /home/p.dalton/test-lk
ls      3138 p.dalton  rtd       DIR    8,1     4096      2 /
ls      3138 p.dalton  txt       REG    8,1  6597697 258133 /usr/local/bin/ls
ls      3138 p.dalton  DEL       REG    8,1            3453 /usr/lib/x86_64-linux-gnu/libc-2.31.so
ls      3138 p.dalton  DEL       REG    8,1            3466 /usr/lib/x86_64-linux-gnu/libpthread-2.31.so
ls      3138 p.dalton  DEL       REG    8,1            3449 /usr/lib/x86_64-linux-gnu/ld-2.31.so
ls      3138 p.dalton    0r      CHR    1,3      0t0      6 /dev/null
ls      3138 p.dalton    1w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log
ls      3138 p.dalton    2w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log
ls      3138 p.dalton    3r      CHR    1,3      0t0      6 /dev/null
ls      3138 p.dalton    4u      REG    8,1        4 258

### Download any suspicious files, binaries, or log files

In [99]:
# Start SFTP session
sftp = ssh.open_sftp()
# List of files to download
files = ['/home/p.dalton/test-lk/.d.log', '/home/p.dalton/test-lk/d.pid']
# Directory to place files (with a z for the kool kidzzz)
downloadDir = "filez"
# DL files
downloadFiles(files,downloadDir)

### Check running processes, /etc/passwd, and /var/log/auth.log

In [100]:
# Start SFTP session
sftp = ssh.open_sftp()
# Check running processes, check sus directory and download.
command = ['sudo -S ps aux','sudo -S cp /var/log/auth.log .', 'sudo -S chmod +r auth.log', 'dir /home/p.dalton/']
# File for output
output = "running+R.txt"
# Run CMDs
runCMDList(command,thePass,output)
# List of files to download
files = ['/etc/passwd', 'auth.log','ps-aux-results.txt', '/home/p.dalton/aqwerpuqwerj']
# Directory to place files (with a z for the kool kidzzz)
downloadDir = "filez"
# DL files
downloadFiles(files,downloadDir)


### BEGIN sudo -S ps aux ###

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.6 103332 103320 ?       Ss   Feb26   0:35 /sbin/init
root           2  0.0  0.0      0     0 ?        S    Feb26   0:00 [kthreadd]
root           3  0.0  0.0      0     0 ?        I<   Feb26   0:00 [rcu_gp]
root           4  0.0  0.0      0     0 ?        I<   Feb26   0:00 [rcu_par_gp]
root           6  0.0  0.0      0     0 ?        I<   Feb26   0:00 [kworker/0:0H-kblockd]
root           8  0.0  0.0      0     0 ?        I<   Feb26   0:00 [mm_percpu_wq]
root           9  0.0  0.0      0     0 ?        S    Feb26   0:02 [ksoftirqd/0]
root          10  0.0  0.0      0     0 ?        I    Feb26   0:34 [rcu_sched]
root          11  0.0  0.0      0     0 ?        S    Feb26   0:07 [migration/0]
root          12  0.0  0.0      0     0 ?        S    Feb26   0:00 [idle_inject/0]
root          14  0.0  0.0      0     0 ?        S    Feb26   0:00 [cpuhp/0]
root      

FileNotFoundError: [Errno 2] No such file

In [ ]:
# close the ssh session
ssh.close()